In [1]:
import os
import random
import pathlib
import tensorflow as tf
import tensorflow.io as tfio
import tensorflow_hub as hub
import matplotlib.pyplot as plt
from tensorflow.keras import datasets, layers, models

In [2]:
AUTOTUNE = tf.data.experimental.AUTOTUNE
NUM_CLASSES = 2
RESIZE_TO = 512
CROP_TO = 480
IMAGE_SIZE = (CROP_TO, CROP_TO)
NUM_CHANNELS = 3
SCHEDULE_LENGTH = 1000
SCHEDULE_BOUNDARIES = [300, 600, 900]
BATCH_SIZE = 126
SCHEDULE_LENGTH = SCHEDULE_LENGTH * 512 / BATCH_SIZE
STEPS_PER_EPOCH = 10
lr = 0.03 * BATCH_SIZE / 512 

def load_and_preprocess_image(path, target_image_shape=IMAGE_SIZE, num_channnels=NUM_CHANNELS):
    image = tf.io.read_file(path)
    image = tf.image.decode_jpeg(image, channels=num_channnels)
    image = tf.image.random_flip_left_right(image)
    image = tf.image.random_flip_up_down(image)
    image = tf.image.random_brightness(image, 0.2, seed=None)
    image = tf.image.random_hue(image, 0.2)
#     image = tf.image.resize(image, [RESIZE_TO, RESIZE_TO])
#     image = tf.image.random_crop(image, [CROP_TO, CROP_TO, 3])
    image = tf.image.resize(image, [CROP_TO, CROP_TO])
    image /= 255.0  # normalize to [0,1] range
#     image = 2*image-1
    return image

class MyBiTModel(tf.keras.Model):
    """BiT with a new head."""

    def __init__(self, num_classes, module):
        super().__init__()

        self.num_classes = num_classes
        self.head = tf.keras.layers.Dense(1, kernel_initializer='zeros', activation='sigmoid')
        self.bit_model = module

    def call(self, images):
        # No need to cut head off since we are using feature extractor model
        bit_embedding = self.bit_model(images)
        return self.head(bit_embedding)

In [3]:
train_val_data_root = '../dataset_5_512/train'
train_val_data_root = pathlib.Path(train_val_data_root)
all_image_paths = [str(path) for path in list(train_val_data_root.glob('*/*'))]
label_names = sorted(item.name for item in train_val_data_root.glob('*/') if item.is_dir())
label_to_index = dict((name, index) for index,name in enumerate(label_names))
all_image_labels = [label_to_index[pathlib.Path(path).parent.name] for path in all_image_paths]
image_count = len(all_image_paths)
image_path_ds = tf.data.Dataset.from_tensor_slices(all_image_paths)
image_ds = image_path_ds.map(load_and_preprocess_image, num_parallel_calls=AUTOTUNE)
label_ds = tf.data.Dataset.from_tensor_slices(tf.cast(all_image_labels, tf.int64))
image_label_ds = tf.data.Dataset.zip((image_ds, label_ds))
train_pipeline = (
    image_label_ds
        .cache()
        .apply(tf.data.experimental.shuffle_and_repeat(buffer_size=len(all_image_paths)))
        .batch(BATCH_SIZE)
        .prefetch(buffer_size=AUTOTUNE)
)
train_steps_per_epoch=tf.math.ceil(len(all_image_paths)/BATCH_SIZE).numpy()

Instructions for updating:
Use `tf.data.Dataset.shuffle(buffer_size, seed)` followed by `tf.data.Dataset.repeat(count)`. Static tf.data optimizations will take care of using the fused implementation.


In [4]:
test_data_root = '../dataset_5_512/val'
test_data_root = pathlib.Path(test_data_root)
all_image_paths = [str(path) for path in list(test_data_root.glob('*/*'))]
label_names = sorted(item.name for item in test_data_root.glob('*/') if item.is_dir())
label_to_index = dict((name, index) for index,name in enumerate(label_names))
all_image_labels = [label_to_index[pathlib.Path(path).parent.name] for path in all_image_paths]
image_count = len(all_image_paths)
image_path_ds = tf.data.Dataset.from_tensor_slices(all_image_paths)
image_ds = image_path_ds.map(load_and_preprocess_image, num_parallel_calls=AUTOTUNE)
label_ds = tf.data.Dataset.from_tensor_slices(tf.cast(all_image_labels, tf.int64))
image_label_ds = tf.data.Dataset.zip((image_ds, label_ds))
test_pipeline = (image_label_ds
                    .cache()
                    .apply(tf.data.experimental.shuffle_and_repeat(buffer_size=len(all_image_paths)))
                    .batch(BATCH_SIZE).prefetch(buffer_size=AUTOTUNE))
val_steps_per_epoch=tf.math.ceil(len(all_image_paths)/BATCH_SIZE).numpy()

In [5]:
model_url = "https://tfhub.dev/google/bit/m-r50x1/1"
module = hub.KerasLayer(model_url)
model = MyBiTModel(num_classes=NUM_CLASSES, module=module)

In [6]:
lr_schedule = tf.keras.optimizers.schedules.PiecewiseConstantDecay(
    boundaries=SCHEDULE_BOUNDARIES,
    values=[lr, lr*0.1, lr*0.001, lr*0.0001]
)
optimizer = tf.keras.optimizers.SGD(
    learning_rate=lr_schedule,
    momentum=0.9
)
loss_fn = tf.keras.losses.BinaryCrossentropy(from_logits=True)
model.compile(optimizer=optimizer,
              loss=loss_fn,
              metrics=['accuracy'])

In [7]:
optimizer = tf.keras.optimizers.Adam(
    learning_rate=0.0001, beta_1=0.9, beta_2=0.999, epsilon=1e-07, amsgrad=False,
)
loss_fn = tf.keras.losses.BinaryCrossentropy()
model.compile(
    loss=loss_fn,
    optimizer=optimizer,
    metrics=['accuracy']
)

In [ ]:
model.fit(
    train_pipeline, 
    epochs=1000, 
    steps_per_epoch=train_steps_per_epoch, 
    validation_data=test_pipeline, 
    validation_steps=val_steps_per_epoch
)

Epoch 1/1000
253/253 [==============================] - 418s 2s/step - loss: 0.3246 - accuracy: 0.8812 - val_loss: 0.2466 - val_accuracy: 0.9057
Epoch 2/1000
253/253 [==============================] - 400s 2s/step - loss: 0.2095 - accuracy: 0.9240 - val_loss: 0.2226 - val_accuracy: 0.9111
Epoch 3/1000
253/253 [==============================] - 396s 2s/step - loss: 0.1853 - accuracy: 0.9311 - val_loss: 0.2078 - val_accuracy: 0.9146
Epoch 4/1000
 35/253 [===>..........................] - ETA: 4:47 - loss: 0.1816 - accuracy: 0.9313